In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib

In [40]:
url='https://www.transfermarkt.com/bundesliga/kreuztabelle/wettbewerb/L1/saison_id/2021'
headers = {'Authorization': 'Bearer YOUR_ACCESS_TOKEN',
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [41]:
PROXY_ADDRESS = "185.199.229.156"
PROXY_PORT =7492
PROXY_USERNAME = "zhudofyh"
PROXY_PASSWORD = "k1tm10gcjs3u"

proxy_credential = {"https": f"http://{PROXY_USERNAME}:{PROXY_PASSWORD}@{PROXY_ADDRESS}:{PROXY_PORT}"}

In [42]:
r = requests.get(url, headers =headers , proxies=proxy_credential)
if r.status_code == 200:

    soup=BeautifulSoup(r.content, 'html.parser')

else:

    print('Error:', r.status_code)

In [44]:
soup.prettify()

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <script type="text/javascript">\n   !function () { var e = function () { var e, t = "__tcfapiLocator", a = [], n = window; for (; n;) { try { if (n.frames[t]) { e = n; break } } catch (e) { } if (n === window.top) break; n = n.parent } e || (!function e() { var a = n.document, r = !!n.frames[t]; if (!r) if (a.body) { var i = a.createElement("iframe"); i.style.cssText = "display:none", i.name = t, a.body.appendChild(i) } else setTimeout(e, 5); return !r }(), n.__tcfapi = function () { for (var e, t = arguments.length, n = new Array(t), r = 0; r < t; r++)n[r] = arguments[r]; if (!n.length) return a; if ("setGdprApplies" === n[0]) n.length > 3 && 2 === parseInt(n[1], 10) && "boolean" == typeof n[3] && (e = n[3], "function" == typeof n[2] && n[2]("set", !0)); else if ("ping" === n[0]) { var i = { gdprApplies: e, cmpLoaded: !1, cmpStatus: "stub" }; "function" == typeof n[2] && n[2](i) } else a.push(n) }, n.addEventListener("message", (function 

In [73]:
def href_to_id(link):
  id=link.split('/')[4] 
  return id

In [176]:
host_names=[]
host_ids=[]
host_status=[]
scores=[]
matches_id=[]
matches_date=[]
guest_club=[]
trs=soup.find('table',attrs={'class':'kreuztabelle'}).find_all('tr')
for tr in trs[1:-1]:
    #find host name and host id(club)
    host_name=tr.find('td').find('a')['title'] 
    #host id
    link=tr.find('td').find('a')['href']
    host_id=href_to_id(link) 

    #find all table data
    tds=tr.find_all('td')[1:-1]
    #find status and score of each game

    for td in tds:

        #find match_id and match day and guest_name
        try:
            #guest name
            guest_name=td.find('nobr').find('a')['href'].split('/')[1].split('_')[1].replace('-',' ').title()


            #match ID
            match_id=td.find('a')['id']

            #match day
            matchday=td.find('a')['title'].split(',')[1][4:].lstrip()
        
        #find scores of games
        except:
            pass

        try:
            try:
                score=td.find('nobr').find('a').find('span',attrs={'class':'redtext'}).text
                status='Lost'
       
            except:
                score=td.find('nobr').find('a').find('span',attrs={'class':'greentext'}).text
                status='Win'
        except:
            try:
                score=td.find('nobr').find('a').find('span').text
                status='equal'
            except:
                pass 
        scores.append(score)
        host_names.append(host_name)
        host_ids.append(host_id)
        host_status.append(status)
        matches_id.append(match_id)
        matches_date.append(matchday)
        guest_club.append(guest_name)

In [177]:
info={}
info['host_name']=host_names
info['host_id']=host_ids
info['host_status']=host_status
info['score']=scores
info['match_id']=matches_id
info['match_date']=matches_date
info['guest_club']=guest_club

In [178]:
df=pd.DataFrame(data=info)
df.head()

,host_name,host_id,host_status,score,match_id,match_date,guest_club
0,Bayern Munich,27,equal,1:1,3588156,21.08.2021,Arminia Bielefeld
1,Bayern Munich,27,Win,3:1,3588607,23.04.2022,Borussia Dortmund
2,Bayern Munich,27,equal,1:1,3588508,05.03.2022,Bayer 04 Leverkusen
3,Bayern Munich,27,Win,3:2,3588454,05.02.2022,Rb Leipzig
4,Bayern Munich,27,Win,4:0,3588544,19.03.2022,1 Fc Union Berlin


In [260]:
df['guest_club']=df['guest_club'].replace('Rb Leipzig','RB Leipzig')
df['guest_club']=df['guest_club'].replace('1 Fc Union Berlin','1.FC Union Berlin')
df['guest_club']=df['guest_club'].replace('1 Fc Koln','1.FC Köln')
df['guest_club']=df['guest_club'].replace('Tsg 1899 Hoffenheim','TSG 1899 Hoffenheim')
df['guest_club']=df['guest_club'].replace('Borussia Monchengladbach','Borussia Mönchengladbach')
df['guest_club']=df['guest_club'].replace('Vfl Wolfsburg','VfL Wolfsburg')
df['guest_club']=df['guest_club'].replace('Vfl Bochum','VfL Bochum')
df['guest_club']=df['guest_club'].replace('Fc Augsburg','FC Augsburg') 
df['guest_club']=df['guest_club'].replace('Vfb Stuttgart','VfB Stuttgart')
df['guest_club']=df['guest_club'].replace('Hertha Bsc','Hertha BSC')
df['guest_club']=df['guest_club'].replace('Hertha Bsc','Hertha BSC')
df['guest_club']=df['guest_club'].replace('Spvgg Greuther Furth','SpVgg Greuther Fürth')
df['guest_club']=df['guest_club'].replace('Sc Freiburg','SC Freiburg') 

In [258]:
df['host_name'].unique()

array(['Bayern Munich', 'Borussia Dortmund', 'Bayer 04 Leverkusen',
       'RB Leipzig', '1.FC Union Berlin', 'SC Freiburg', '1.FC Köln',
       '1.FSV Mainz 05', 'TSG 1899 Hoffenheim',
       'Borussia Mönchengladbach', 'Eintracht Frankfurt', 'VfL Wolfsburg',
       'VfL Bochum', 'FC Augsburg', 'VfB Stuttgart', 'Hertha BSC',
       'Arminia Bielefeld', 'SpVgg Greuther Fürth'], dtype=object)

In [259]:
df['guest_club'].unique()

array(['Arminia Bielefeld', 'Borussia Dortmund', 'Bayer 04 Leverkusen',
       'RB Leipzig', '1.FC Union Berlin', 'Sc Freiburg', '1.FC Köln',
       '1.FSV Mainz 05', 'TSG 1899 Hoffenheim',
       'Borussia Mönchengladbach', 'Eintracht Frankfurt', 'VfL Wolfsburg',
       'VfL Bochum', 'FC Augsburg', 'VfB Stuttgart', 'Hertha BSC',
       'SpVgg Greuther Fürth', 'Bayern Munich'], dtype=object)

In [ ]:
df_id=pd.DataFrame(data=df[['host_name','host_id']])

In [210]:
clubs=pd.read_csv('Club_table.csv')

In [211]:
clubs.head()

,id,Club,Country
0,631,Chelsea FC,England
1,281,Manchester City,England
2,11,Arsenal FC,England
3,985,Manchester United,England
4,31,Liverpool FC,England


In [212]:
clubs.drop(['Country'],axis=1,inplace=True)

In [213]:
clubs.shape

(149, 2)

In [214]:
clubs.drop_duplicates(inplace=True,keep='last')

In [215]:
clubs.shape

(149, 2)

In [216]:
clubs.rename(columns={'Club':'guest_club','id':'guest_id'},inplace=True)

In [217]:
clubs.head()

,guest_id,guest_club
0,631,Chelsea FC
1,281,Manchester City
2,11,Arsenal FC
3,985,Manchester United
4,31,Liverpool FC


In [266]:
merged=df.merge(clubs,how='left',on='guest_club')

In [267]:
merged.head()

,host_name,host_id,host_status,score,match_id,match_date,guest_club,guest_id
0,Bayern Munich,27,equal,1:1,3588156,21.08.2021,Arminia Bielefeld,10
1,Bayern Munich,27,Win,3:1,3588607,23.04.2022,Borussia Dortmund,16
2,Bayern Munich,27,equal,1:1,3588508,05.03.2022,Bayer 04 Leverkusen,15
3,Bayern Munich,27,Win,3:2,3588454,05.02.2022,RB Leipzig,23826
4,Bayern Munich,27,Win,4:0,3588544,19.03.2022,1.FC Union Berlin,89


In [268]:
merged.isnull().sum()

host_name      0
host_id        0
host_status    0
score          0
match_id       0
match_date     0
guest_club     0
guest_id       0
dtype: int64

In [269]:
merged.drop(['guest_club','host_name'],axis=1,inplace=True)

In [270]:
merged.head()

,host_id,host_status,score,match_id,match_date,guest_id
0,27,equal,1:1,3588156,21.08.2021,10
1,27,Win,3:1,3588607,23.04.2022,16
2,27,equal,1:1,3588508,05.03.2022,15
3,27,Win,3:2,3588454,05.02.2022,23826
4,27,Win,4:0,3588544,19.03.2022,89


In [274]:
merged.to_csv('Germany_competions.csv', index=False)

#England

In [372]:
url='https://www.transfermarkt.com/premier-league/kreuztabelle/wettbewerb/GB1/saison_id/2021'
headers = {'Authorization': 'Bearer YOUR_ACCESS_TOKEN',
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [373]:
PROXY_ADDRESS = "185.199.229.156"
PROXY_PORT =7492
PROXY_USERNAME = "zhudofyh"
PROXY_PASSWORD = "k1tm10gcjs3u"

proxy_credential = {"https": f"http://{PROXY_USERNAME}:{PROXY_PASSWORD}@{PROXY_ADDRESS}:{PROXY_PORT}"}

In [374]:
r = requests.get(url, headers =headers , proxies=proxy_credential)
if r.status_code == 200:

    soup=BeautifulSoup(r.content, 'html.parser')

else:

    print('Error:', r.status_code) 

In [375]:
host_names=[]
host_ids=[]
host_status=[]
scores=[]
matches_id=[]
matches_date=[]
guest_club=[]
trs=soup.find('table',attrs={'class':'kreuztabelle'}).find_all('tr')
row_1=trs[0].find_all('td')[1:-1]
clubs_count=len(row_1)

for td in row_1:
  guest_id=td.find('a')['href'].split('/')[4]
  guest_club.append(guest_id)

for tr in trs[1:-1]:
    #find host name and host id(club)
    host_name=tr.find('td').find('a')['title'] 
    #host id
    link=tr.find('td').find('a')['href']
    host_id=href_to_id(link) 

    #find all table data
    tds=tr.find_all('td')[1:-1]
    #find status and score of each game

    for td in tds:

        #find match_id and match day and guest_name
        try:

            #match ID
            match_id=td.find('a')['id']

            #match day
            matchday=td.find('a')['title'].split(',')[1][4:].lstrip()

        except:
            pass
        #find scores of games
        try:
            try:
                score=td.find('nobr').find('a').find('span',attrs={'class':'redtext'}).text
                status='Lost'
       
            except:
                score=td.find('nobr').find('a').find('span',attrs={'class':'greentext'}).text
                status='Win'
        except:
            try:
                score=td.find('nobr').find('a').find('span').text
                status='equal'
            except:
                score='-' 
                status='-'
        scores.append(score)
        host_names.append(host_name)
        host_ids.append(host_id)
        host_status.append(status)
        matches_id.append(match_id)
        matches_date.append(matchday)

In [376]:
guest_club=guest_club*clubs_count

In [377]:
info={}
info['host_name']=host_names
info['host_id']=host_ids
info['host_status']=host_status
info['score']=scores
info['match_id']=matches_id
info['match_date']=matches_date
info['guest_club']=guest_club

In [378]:
england_df=pd.DataFrame(data=info)

In [379]:
england_df.shape

(400, 7)

In [380]:
england_df.head()

,host_name,host_id,host_status,score,match_id,match_date,guest_club
0,Manchester City,281,-,-,3607541,06.11.2021,281
1,Manchester City,281,equal,2:2,3592215,10.04.2022,31
2,Manchester City,281,Win,1:0,3592115,15.01.2022,631
3,Manchester City,281,Lost,2:3,3592107,19.02.2022,148
4,Manchester City,281,Win,5:0,3591995,28.08.2021,11


In [381]:
index=england_df[england_df['score']=='-'].index.tolist()

In [382]:
england_df.drop(index,axis=0,inplace=True)

In [383]:
england_df.shape

(380, 7)

In [384]:
england_df.head(30)

,host_name,host_id,host_status,score,match_id,match_date,guest_club
1,Manchester City,281,equal,2:2,3592215,10.04.2022,31
2,Manchester City,281,Win,1:0,3592115,15.01.2022,631
3,Manchester City,281,Lost,2:3,3592107,19.02.2022,148
4,Manchester City,281,Win,5:0,3591995,28.08.2021,11
5,Manchester City,281,Win,4:1,3592145,06.03.2022,985
6,Manchester City,281,Win,2:1,3592099,28.11.2021,379
7,Manchester City,281,Win,6:3,3592208,26.12.2021,1003
8,Manchester City,281,Win,3:0,3592217,20.04.2022,1237
9,Manchester City,281,Win,1:0,3592079,11.12.2021,543
10,Manchester City,281,Win,5:0,3592265,08.05.2022,762


In [385]:
england_df.isnull().sum()

host_name      0
host_id        0
host_status    0
score          0
match_id       0
match_date     0
guest_club     0
dtype: int64

In [387]:
england_df.drop(['host_name'],axis=1,inplace=True)

In [388]:
england_df.to_csv('england_competions.csv', index=False)

#Italy

In [402]:
url='https://www.transfermarkt.com/serie-a/kreuztabelle/wettbewerb/IT1/saison_id/2021'

In [403]:
r = requests.get(url, headers =headers , proxies=proxy_credential)
if r.status_code == 200:

    soup=BeautifulSoup(r.content, 'html.parser')

else:

    print('Error:', r.status_code) 

In [404]:
host_names=[]
host_ids=[]
host_status=[]
scores=[]
matches_id=[]
matches_date=[]
guest_club=[]
trs=soup.find('table',attrs={'class':'kreuztabelle'}).find_all('tr')
row_1=trs[0].find_all('td')[1:-1]
clubs_count=len(row_1)

for td in row_1:
  guest_id=td.find('a')['href'].split('/')[4]
  guest_club.append(guest_id)

for tr in trs[1:-1]:
    #find host name and host id(club)
    host_name=tr.find('td').find('a')['title'] 
    #host id
    link=tr.find('td').find('a')['href']
    host_id=href_to_id(link) 

    #find all table data
    tds=tr.find_all('td')[1:-1]
    #find status and score of each game

    for td in tds:

        #find match_id and match day and guest_name
        try:

            #match ID
            match_id=td.find('a')['id']

            #match day
            matchday=td.find('a')['title'].split(',')[1][4:].lstrip()

        except:
            pass
        #find scores of games
        try:
            try:
                score=td.find('nobr').find('a').find('span',attrs={'class':'redtext'}).text
                status='Lost'
       
            except:
                score=td.find('nobr').find('a').find('span',attrs={'class':'greentext'}).text
                status='Win'
        except:
            try:
                score=td.find('nobr').find('a').find('span').text
                status='equal'
            except:
                score='-' 
                status='-'
        scores.append(score)
        host_names.append(host_name)
        host_ids.append(host_id)
        host_status.append(status)
        matches_id.append(match_id)
        matches_date.append(matchday)

In [405]:
guest_club=guest_club*clubs_count

In [406]:
info={}
info['host_name']=host_names
info['host_id']=host_ids
info['host_status']=host_status
info['score']=scores
info['match_id']=matches_id
info['match_date']=matches_date
info['guest_club']=guest_club

In [407]:
italy_df=pd.DataFrame(data=info)

In [408]:
italy_df.shape

(400, 7)

In [409]:
italy_df.head()

,host_name,host_id,host_status,score,match_id,match_date,guest_club
0,AC Milan,5,-,-,3589498,10.04.2022,5
1,AC Milan,5,equal,1:1,3615120,07.11.2021,46
2,AC Milan,5,Lost,0:1,3614713,19.12.2021,6195
3,AC Milan,5,equal,0:0,3614754,23.01.2022,506
4,AC Milan,5,Win,2:0,3614545,12.09.2021,398


In [410]:
index=italy_df[italy_df['score']=='-'].index.tolist()

In [411]:
italy_df.drop(index,axis=0,inplace=True)

In [412]:
italy_df.shape

(380, 7)

In [420]:
italy.isnull().sum()

host_name      0
host_id        0
host_status    0
score          0
match_id       0
match_date     0
guest_club     0
dtype: int64

In [413]:
italy_df.to_csv('Italy_competions.csv', index=False)

#final DataFrame

In [428]:
france=pd.read_csv('France_competions.csv')
germany=pd.read_csv('Germany_competions.csv')
italy=pd.read_csv('Italy_competions.csv')
england=pd.read_csv('england_competions.csv')
spain=pd.read_csv('spain_competions.csv')

In [429]:
france.isnull().sum()

host_name      0
host_id        0
host_status    0
score          0
match_id       0
match_date     0
guest_club     0
dtype: int64

In [430]:
france.drop(['host_name'],axis=1,inplace=True)

In [449]:
france.rename(columns={'guest_club':'guest_id'},inplace=True)

In [450]:
france.head()

,host_id,host_status,score,match_id,match_date,guest_id
0,583,Win,2:1,3589530,17.04.2022,244
1,583,Win,2:0,3589292,12.12.2021,162
2,583,Win,1:0,3589388,11.02.2022,273
3,583,equal,0:0,3589262,01.12.2021,417
4,583,Win,4:2,3588789,14.08.2021,667


In [451]:
germany.isnull().sum()

host_id        0
host_status    0
score          0
match_id       0
match_date     0
guest_id       0
dtype: int64

In [432]:
germany 

,host_id,host_status,score,match_id,match_date,guest_id
0,27,equal,1:1,3588156,21.08.2021,10
1,27,Win,3:1,3588607,23.04.2022,16
2,27,equal,1:1,3588508,05.03.2022,15
3,27,Win,3:2,3588454,05.02.2022,23826
4,27,Win,4:0,3588544,19.03.2022,89
...,...,...,...,...,...,...
319,65,equal,0:0,3588390,18.12.2021,167
320,65,equal,0:0,3588408,08.01.2022,79
321,65,Win,2:1,3588471,12.02.2022,44
322,65,equal,1:1,3588156,21.08.2021,10


In [452]:
italy.isnull().sum()

host_id        0
host_status    0
score          0
match_id       0
match_date     0
guest_club     0
dtype: int64

In [453]:
italy.rename(columns={'guest_club':'guest_id'},inplace=True)

In [434]:
italy.drop(['host_name'],axis=1,inplace=True)

In [454]:
italy.head()

,host_id,host_status,score,match_id,match_date,guest_id
0,5,equal,1:1,3615120,07.11.2021,46
1,5,Lost,0:1,3614713,19.12.2021,6195
2,5,equal,0:0,3614754,23.01.2022,506
3,5,Win,2:0,3614545,12.09.2021,398
4,5,Win,3:1,3614523,06.01.2022,12


In [455]:
england.isnull().sum()

host_id        0
host_status    0
score          0
match_id       0
match_date     0
guest_club     0
dtype: int64

In [456]:
england.rename(columns={'guest_club':'guest_id'},inplace=True)

In [437]:
spain.isnull().sum()

host_id        0
host_status    0
score          0
match_id       0
match_date     0
guest_id       0
dtype: int64

In [457]:
merged=pd.concat([france,germany,italy,england,spain],ignore_index=True)

In [458]:
merged.shape

(1788, 6)

In [463]:
merged['Season']=2021

In [464]:
merged.head()

,host_id,host_status,score,match_id,match_date,guest_id,Season
0,583,Win,2:1,3589530,17.04.2022,244,2021
1,583,Win,2:0,3589292,12.12.2021,162,2021
2,583,Win,1:0,3589388,11.02.2022,273,2021
3,583,equal,0:0,3589262,01.12.2021,417,2021
4,583,Win,4:2,3588789,14.08.2021,667,2021


In [465]:
merged.isnull().sum()

host_id        0
host_status    0
score          0
match_id       0
match_date     0
guest_id       0
Season         0
dtype: int64

In [476]:
merged.drop_duplicates(keep='last',inplace=True)

In [470]:
merged['goals_for']=merged['score'].apply(lambda x: x.split(':')[0])

In [473]:
merged['goals_against']=merged['score'].apply(lambda x: x.split(':')[1])

In [478]:
merged['match_date']=pd.to_datetime(merged['match_date'],format='%d.%m.%Y')

In [479]:
merged

,host_id,host_status,score,match_id,match_date,guest_id,Season,goals_for,goals_against
0,583,Win,2:1,3589530,2022-04-17,244,2021,2,1
1,583,Win,2:0,3589292,2021-12-12,162,2021,2,0
2,583,Win,1:0,3589388,2022-02-11,273,2021,1,0
3,583,equal,0:0,3589262,2021-12-01,417,2021,0,0
4,583,Win,4:2,3588789,2021-08-14,667,2021,4,2
...,...,...,...,...,...,...,...,...,...
1782,65,Lost,0:1,3588246,2021-10-16,80,2021,0,1
1783,65,equal,0:0,3588390,2021-12-18,167,2021,0,0
1784,65,equal,0:0,3588408,2022-01-08,79,2021,0,0
1785,65,Win,2:1,3588471,2022-02-12,44,2021,2,1


In [480]:
merged.to_csv('competions.csv', index=False)